In [1]:
# Libraries
import pandas as pd
import regex as re
import nltk
from nltk.classify import SklearnClassifier
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /Users/Temi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/Temi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/Temi/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
df = pd.read_pickle('cleaned_steam_data_V2', compression='infer')
data = df.copy()
data.head()

,type,name,steam_appid,about_the_game,short_description,developers,publishers,categories,genres,release_date
0,game,Counter-Strike,10,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,[Valve],[Valve],"[Multi-player, PvP, Online PvP, Shared/Split S...",[Action],1 Nov 2000
1,game,Team Fortress Classic,20,One of the most popular online action games of...,One of the most popular online action games of...,[Valve],[Valve],"[Multi-player, PvP, Online PvP, Shared/Split S...",[Action],1 Apr 1999
2,game,Day of Defeat,30,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,[Valve],[Valve],"[Multi-player, Valve Anti-Cheat enabled]",[Action],1 May 2003
3,game,Deathmatch Classic,40,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,[Valve],[Valve],"[Multi-player, PvP, Online PvP, Shared/Split S...",[Action],1 Jun 2001
4,game,Half-Life: Opposing Force,50,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,[Gearbox Software],[Valve],"[Single-player, Multi-player, Valve Anti-Cheat...",[Action],1 Nov 1999


In [3]:
#function to replace the date that starts with 0 to 1 
def date_fixer(row):
    if (str(row.release_date).startswith('0')):
        return str(row.release_date).replace('0', '1', 1)
    else: return str(row.release_date)

#applying the above function
data['new_release_date'] = data.apply(date_fixer, axis=1)
data.head()

,type,name,steam_appid,about_the_game,short_description,developers,publishers,categories,genres,release_date,new_release_date
0,game,Counter-Strike,10,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,[Valve],[Valve],"[Multi-player, PvP, Online PvP, Shared/Split S...",[Action],1 Nov 2000,1 Nov 2000
1,game,Team Fortress Classic,20,One of the most popular online action games of...,One of the most popular online action games of...,[Valve],[Valve],"[Multi-player, PvP, Online PvP, Shared/Split S...",[Action],1 Apr 1999,1 Apr 1999
2,game,Day of Defeat,30,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,[Valve],[Valve],"[Multi-player, Valve Anti-Cheat enabled]",[Action],1 May 2003,1 May 2003
3,game,Deathmatch Classic,40,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,[Valve],[Valve],"[Multi-player, PvP, Online PvP, Shared/Split S...",[Action],1 Jun 2001,1 Jun 2001
4,game,Half-Life: Opposing Force,50,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,[Gearbox Software],[Valve],"[Single-player, Multi-player, Valve Anti-Cheat...",[Action],1 Nov 1999,1 Nov 1999


In [4]:
#just to test, dropping the coming_soon for now or we may omit this process 
data= data.drop(data[(data['new_release_date'] == 'coming_soon')].index)
data.head()

,type,name,steam_appid,about_the_game,short_description,developers,publishers,categories,genres,release_date,new_release_date
0,game,Counter-Strike,10,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,[Valve],[Valve],"[Multi-player, PvP, Online PvP, Shared/Split S...",[Action],1 Nov 2000,1 Nov 2000
1,game,Team Fortress Classic,20,One of the most popular online action games of...,One of the most popular online action games of...,[Valve],[Valve],"[Multi-player, PvP, Online PvP, Shared/Split S...",[Action],1 Apr 1999,1 Apr 1999
2,game,Day of Defeat,30,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,[Valve],[Valve],"[Multi-player, Valve Anti-Cheat enabled]",[Action],1 May 2003,1 May 2003
3,game,Deathmatch Classic,40,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,[Valve],[Valve],"[Multi-player, PvP, Online PvP, Shared/Split S...",[Action],1 Jun 2001,1 Jun 2001
4,game,Half-Life: Opposing Force,50,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,[Gearbox Software],[Valve],"[Single-player, Multi-player, Valve Anti-Cheat...",[Action],1 Nov 1999,1 Nov 1999


In [5]:
#dropping the old release date as we have cleaned the date
data = data.drop(columns =['release_date'])
data.head()

,type,name,steam_appid,about_the_game,short_description,developers,publishers,categories,genres,new_release_date
0,game,Counter-Strike,10,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,[Valve],[Valve],"[Multi-player, PvP, Online PvP, Shared/Split S...",[Action],1 Nov 2000
1,game,Team Fortress Classic,20,One of the most popular online action games of...,One of the most popular online action games of...,[Valve],[Valve],"[Multi-player, PvP, Online PvP, Shared/Split S...",[Action],1 Apr 1999
2,game,Day of Defeat,30,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,[Valve],[Valve],"[Multi-player, Valve Anti-Cheat enabled]",[Action],1 May 2003
3,game,Deathmatch Classic,40,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,[Valve],[Valve],"[Multi-player, PvP, Online PvP, Shared/Split S...",[Action],1 Jun 2001
4,game,Half-Life: Opposing Force,50,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,[Gearbox Software],[Valve],"[Single-player, Multi-player, Valve Anti-Cheat...",[Action],1 Nov 1999


In [6]:
# Function to remove punctuation
def remove_punctuation(text):
    no_punc = ''.join([c for c in text if c not in string.punctuation])
    return no_punc

# Remove punctuation from long description columns
data['about_the_game'] = data['about_the_game'].apply(lambda x: remove_punctuation(x))
data['short_description'] = data['short_description'].apply(lambda x: remove_punctuation(x))
data.head()

,type,name,steam_appid,about_the_game,short_description,developers,publishers,categories,genres,new_release_date
0,game,Counter-Strike,10,Play the worlds number 1 online action game En...,Play the worlds number 1 online action game En...,[Valve],[Valve],"[Multi-player, PvP, Online PvP, Shared/Split S...",[Action],1 Nov 2000
1,game,Team Fortress Classic,20,One of the most popular online action games of...,One of the most popular online action games of...,[Valve],[Valve],"[Multi-player, PvP, Online PvP, Shared/Split S...",[Action],1 Apr 1999
2,game,Day of Defeat,30,Enlist in an intense brand of Axis vs Allied t...,Enlist in an intense brand of Axis vs Allied t...,[Valve],[Valve],"[Multi-player, Valve Anti-Cheat enabled]",[Action],1 May 2003
3,game,Deathmatch Classic,40,Enjoy fastpaced multiplayer gaming with Deathm...,Enjoy fastpaced multiplayer gaming with Deathm...,[Valve],[Valve],"[Multi-player, PvP, Online PvP, Shared/Split S...",[Action],1 Jun 2001
4,game,Half-Life: Opposing Force,50,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,[Gearbox Software],[Valve],"[Single-player, Multi-player, Valve Anti-Cheat...",[Action],1 Nov 1999


In [7]:
# Make descriptions lower case
data['about_the_game']= data['about_the_game'].apply(lambda x: x.lower())
data['short_description']= data['short_description'].apply(lambda x: x.lower())
data.head()

,type,name,steam_appid,about_the_game,short_description,developers,publishers,categories,genres,new_release_date
0,game,Counter-Strike,10,play the worlds number 1 online action game en...,play the worlds number 1 online action game en...,[Valve],[Valve],"[Multi-player, PvP, Online PvP, Shared/Split S...",[Action],1 Nov 2000
1,game,Team Fortress Classic,20,one of the most popular online action games of...,one of the most popular online action games of...,[Valve],[Valve],"[Multi-player, PvP, Online PvP, Shared/Split S...",[Action],1 Apr 1999
2,game,Day of Defeat,30,enlist in an intense brand of axis vs allied t...,enlist in an intense brand of axis vs allied t...,[Valve],[Valve],"[Multi-player, Valve Anti-Cheat enabled]",[Action],1 May 2003
3,game,Deathmatch Classic,40,enjoy fastpaced multiplayer gaming with deathm...,enjoy fastpaced multiplayer gaming with deathm...,[Valve],[Valve],"[Multi-player, PvP, Online PvP, Shared/Split S...",[Action],1 Jun 2001
4,game,Half-Life: Opposing Force,50,return to the black mesa research facility as ...,return to the black mesa research facility as ...,[Gearbox Software],[Valve],"[Single-player, Multi-player, Valve Anti-Cheat...",[Action],1 Nov 1999


In [8]:
# Drop square brackets from developers, publishers, categories and genres columns
data['developers'] = data['developers'].str.get(0)
data['publishers'] = data['publishers'].str.get(0)
#data['categories'] = data['categories'].str.get(0) # Finding alternative as this removes all categories after 1st
data['genres'] = data['genres'].str.get(0)

# Remove hyphen from categories column
#data['categories'] = data['categories'].replace('-', ' ', regex=True).astype(str)
data.head()

,type,name,steam_appid,about_the_game,short_description,developers,publishers,categories,genres,new_release_date
0,game,Counter-Strike,10,play the worlds number 1 online action game en...,play the worlds number 1 online action game en...,Valve,Valve,"[Multi-player, PvP, Online PvP, Shared/Split S...",Action,1 Nov 2000
1,game,Team Fortress Classic,20,one of the most popular online action games of...,one of the most popular online action games of...,Valve,Valve,"[Multi-player, PvP, Online PvP, Shared/Split S...",Action,1 Apr 1999
2,game,Day of Defeat,30,enlist in an intense brand of axis vs allied t...,enlist in an intense brand of axis vs allied t...,Valve,Valve,"[Multi-player, Valve Anti-Cheat enabled]",Action,1 May 2003
3,game,Deathmatch Classic,40,enjoy fastpaced multiplayer gaming with deathm...,enjoy fastpaced multiplayer gaming with deathm...,Valve,Valve,"[Multi-player, PvP, Online PvP, Shared/Split S...",Action,1 Jun 2001
4,game,Half-Life: Opposing Force,50,return to the black mesa research facility as ...,return to the black mesa research facility as ...,Gearbox Software,Valve,"[Single-player, Multi-player, Valve Anti-Cheat...",Action,1 Nov 1999


In [9]:
# Remove stopwords
#stop_words = nltk.corpus.stopwords.words('english')
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    #no_stopwords = [i for i in text if i not in stopwords]
    tokens = word_tokenize(text)
    result = [i for i in tokens if not i in stop_words]
    return result

data['about_the_game'] = data['about_the_game'].apply(lambda x: remove_stopwords(x))
data['short_description'] = data['short_description'].apply(lambda x: remove_stopwords(x))
data.head()

,type,name,steam_appid,about_the_game,short_description,developers,publishers,categories,genres,new_release_date
0,game,Counter-Strike,10,"[play, worlds, number, 1, online, action, game...","[play, worlds, number, 1, online, action, game...",Valve,Valve,"[Multi-player, PvP, Online PvP, Shared/Split S...",Action,1 Nov 2000
1,game,Team Fortress Classic,20,"[one, popular, online, action, games, time, te...","[one, popular, online, action, games, time, te...",Valve,Valve,"[Multi-player, PvP, Online PvP, Shared/Split S...",Action,1 Apr 1999
2,game,Day of Defeat,30,"[enlist, intense, brand, axis, vs, allied, tea...","[enlist, intense, brand, axis, vs, allied, tea...",Valve,Valve,"[Multi-player, Valve Anti-Cheat enabled]",Action,1 May 2003
3,game,Deathmatch Classic,40,"[enjoy, fastpaced, multiplayer, gaming, deathm...","[enjoy, fastpaced, multiplayer, gaming, deathm...",Valve,Valve,"[Multi-player, PvP, Online PvP, Shared/Split S...",Action,1 Jun 2001
4,game,Half-Life: Opposing Force,50,"[return, black, mesa, research, facility, one,...","[return, black, mesa, research, facility, one,...",Gearbox Software,Valve,"[Single-player, Multi-player, Valve Anti-Cheat...",Action,1 Nov 1999


In [10]:
# Lemmatization
wordnet_lemmatizer = WordNetLemmatizer()

def lemmatizer(text):
    lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
    return lemm_text

data['about_the_game'] = data['about_the_game'].apply(lambda x: lemmatizer(x))
data['short_description'] = data['short_description'].apply(lambda x: lemmatizer(x))
data.head()

,type,name,steam_appid,about_the_game,short_description,developers,publishers,categories,genres,new_release_date
0,game,Counter-Strike,10,"[play, world, number, 1, online, action, game,...","[play, world, number, 1, online, action, game,...",Valve,Valve,"[Multi-player, PvP, Online PvP, Shared/Split S...",Action,1 Nov 2000
1,game,Team Fortress Classic,20,"[one, popular, online, action, game, time, tea...","[one, popular, online, action, game, time, tea...",Valve,Valve,"[Multi-player, PvP, Online PvP, Shared/Split S...",Action,1 Apr 1999
2,game,Day of Defeat,30,"[enlist, intense, brand, axis, v, allied, team...","[enlist, intense, brand, axis, v, allied, team...",Valve,Valve,"[Multi-player, Valve Anti-Cheat enabled]",Action,1 May 2003
3,game,Deathmatch Classic,40,"[enjoy, fastpaced, multiplayer, gaming, deathm...","[enjoy, fastpaced, multiplayer, gaming, deathm...",Valve,Valve,"[Multi-player, PvP, Online PvP, Shared/Split S...",Action,1 Jun 2001
4,game,Half-Life: Opposing Force,50,"[return, black, mesa, research, facility, one,...","[return, black, mesa, research, facility, one,...",Gearbox Software,Valve,"[Single-player, Multi-player, Valve Anti-Cheat...",Action,1 Nov 1999
